In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
from typing import List

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [8]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [11]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())

joke = parser.parse(output)

In [14]:
print(joke.setup)
print(joke.punchline)

Why did the chicken cross the road?
To get to the other side!


In [29]:
from langchain.output_parsers import OutputFixingParser
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

class Distance(BaseModel):
    dist: List[float] = Field(
        description="distance between two points",
        min_items=3,
        max_items=3,
    )

    @validator("dist")
    def distance_is_positive(cls, field):
        if any([x < 0 for x in field]):
            raise ValueError("Distance must be positive!")
        return field

parser = PydanticOutputParser(pydantic_object=Distance)
new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

# prompt = PromptTemplate(
#     template="Answer the user query.\n{format_instructions}\n{query}\n",
#     input_variables=["query"],
#     partial_variables={
#         "format_instructions": parser.get_format_instructions()},
# )

examples = [
    {"input":"woman close fornt lemon tree\nlocal position:",
        "output":" [0.44,0.00,1.45]\n"},
    {"input": "woman close left lemon tree\nlocal position:",
        "output": " [-2.47,0.00,0.23]\n"},
    {"input": "woman close right lemon tree\nlocal position:",
        "output": " [1.75,0.00,-0.42]\n"},
    {"input": "woman close behind lemon tree\nlocal position:",
        "output": " [-0.28,0.00,-2.22]\n"}
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

# _input = prompt.format_prompt(query="What is the distance between (1, 2, 3) and (4, 5, 6)?")

# output = model(_input.to_string())

# distance = new_parser.parse(output)

Human: woman close fornt lemon tree
local position:
AI:  [0.44,0.00,1.45]

Human: woman close left lemon tree
local position:
AI:  [-2.47,0.00,0.23]

Human: woman close right lemon tree
local position:
AI:  [1.75,0.00,-0.42]

Human: woman close behind lemon tree
local position:
AI:  [-0.28,0.00,-2.22]



In [28]:
distance

Distance(dist=[1.0, 2.0, 3.0])